In [ ]:
import time
import argparse
import torch
import msgpack
import spacy
import json
import re
import unicodedata
import collections
import pandas as pd
import numpy as np

In [ ]:
def flatten_json(data_file, mode):
    """Flatten each article in training data."""
    with open(data_file) as f:
        data = json.load(f)['data']
    rows = []
    for article in data:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                id_, question, answers = qa['id'], qa['question'], qa['answers']
                if mode == 'train':
                    answer = answers[0]['text']  # in training data there's only one answer
                    answer_start = answers[0]['answer_start'] # char level length
                    answer_end = answer_start + len(answer) # char level lenght
                    rows.append((id_, context, question, answer, answer_start, answer_end))
                else:  # mode == 'dev'
                    answers = [a['text'] for a in answers]
                    rows.append((id_, context, question, answers))
    return rows

def iob_np_tag(tag_list):
    '''
    function for creating iob_np
    @in: a list of POS tags
    @out: iob_np tags
    '''
    iob_np = ['o_np'] * len(tag_list)
    for i in range(len(tag_list)):
        if 'NN' in tag_list[i]:
            if iob_np[i-1] == 'b_np':
                iob_np[i] = 'i_np'
            elif iob_np[i-1] == 'i_np':
                iob_np[i] = 'i_np'
            else:
                iob_np[i] = 'b_np'
        i +=1
    return iob_np

## iob tag for NER
def iob_ner_tag(tag_list):
    '''
    function for creating iob_ner
    @in: a list of ner tags
    @out: iob_ner tags
    '''
    iob_ner = ['o_ner'] * len(tag_list)
    for i in range(len(tag_list)):
        if len(tag_list[i]) != 0:
            if iob_ner[i-1] == 'b_ner':
                iob_ner[i] = 'i_ner'
            elif iob_ner[i-1] == 'i_ner':
                iob_ner[i] = 'i_ner'
            else:
                iob_ner[i] = 'b_ner'
        i +=1
    return iob_ner

## Part of NER tag
stop_words = ['a', 'an', 'the', 'of', 'for', '\'s', 'For', 'The', 'A', 'An', ',', ':', '.', ' ,', ', ']
def part_ner_tag(tag_list, context_list):
    '''
    @in: a list of ner tags
    @out: part of ner tags
    '''
    ner_context = []
    part_ner = ['o_ner'] * len(tag_list)
    for i in range(len(tag_list)):
        if len(tag_list[i]) != 0 and context_list[i] not in stop_words:
            part_ner[i] = 'i_ner'
            ner_context.append(context_list[i])

    # combine lemma to ner_context list
    ner_context_str = ' '.join(ner_context)
    ner_context_ = nlp(ner_context_str)
    ner_context_lemma = [w.lemma_ if w.lemma_ != '-PRON-' else w.text.lower() for w in ner_context_]
    ner_context_all = ner_context_lemma +  ner_context

    for j in range(len(context_list)):
        if context_list[j] in ner_context_all:
            part_ner[j] = 'i_ner'
    return part_ner

def clean_spaces(text):
    """normalize spaces in a string."""
    text = re.sub(r'\s', ' ', text)
    return text

def normalize_text(text):
    return unicodedata.normalize('NFD', text)

nlp = None

def init():
    """initialize spacy in each process"""
    '''
    'en': Noun chunks are "base noun phrases" – flat phrases that have a noun as their head.
    parser=False or disable=['parser'] : don't need any of the syntactic information,
                                        and will make spaCy load and run much faster.
    '''
    global nlp
    nlp = spacy.load('en', parser=False)

def annotate(row):
    '''
    notice: the tagging feature only apply on context
    '''
    global nlp
    id_, context, question = row[:3]
    q_doc = nlp(clean_spaces(question))
    c_doc = nlp(clean_spaces(context))
    question_tokens = [normalize_text(w.text) for w in q_doc]
    context_tokens = [normalize_text(w.text) for w in c_doc]
    question_tokens_lower = [w.lower() for w in question_tokens]
    context_tokens_lower = [w.lower() for w in context_tokens]
    context_token_span = [(w.idx, w.idx + len(w.text)) for w in c_doc] # the lenghth of each tokens
    #for context the new features
    context_tags = [w.tag_ for w in c_doc] # POS tagging
    context_ents = [w.ent_type_ for w in c_doc] # NER tagging
    context_iob_np = iob_np_tag(context_tags) # iob_np
    context_iob_ner = iob_ner_tag(context_ents) #iob_ner
    context_part_ner = part_ner_tag(context_ents, context_tokens) #part_ner

    #for question the new features
    question_tags = [w.tag_ for w in q_doc] # POS tagging
    question_ents = [w.ent_type_ for w in q_doc] # NER tagging
    question_iob_np = iob_np_tag(question_tags) # iob_np
    question_iob_ner = iob_ner_tag(question_ents) #iob_ner

    question_lemma = {w.lemma_ if w.lemma_ != '-PRON-' else w.text.lower() for w in q_doc}
    # PRON is such as me/it/you
    # lemma_ : cats -> cat

    question_tokens_set = set(question_tokens)
    question_tokens_lower_set = set(question_tokens_lower)
    match_origin = [w in question_tokens_set for w in context_tokens]
    match_lower = [w in question_tokens_lower_set for w in context_tokens_lower]
    match_lemma = [(w.lemma_ if w.lemma_ != '-PRON-' else w.text.lower()) in question_lemma for w in c_doc]
    # term frequency in document
    counter_ = collections.Counter(context_tokens_lower)
    total = len(context_tokens_lower)
    # frequent feature
    context_tf = [counter_[w] / total for w in context_tokens_lower]
    # exact match feature refering to the paper
    context_features = list(zip(match_origin, match_lower, match_lemma, context_tf))
    
    context_tokens = context_tokens_lower
    question_tokens = question_tokens_lower
    return (id_, context_tokens, context_features, context_tags, context_ents, context_iob_np, context_iob_ner, context_part_ner,
            question_tags,question_ents,question_iob_np,question_iob_ner,
            question_tokens, context, context_token_span) + row[3:]

def index_answer(row):
    token_span = row[-4] #context_token_span
    starts, ends = zip(*token_span)
    answer_start = row[-2]
    answer_end = row[-1]
    try:
        return row[:-3] + (starts.index(answer_start), ends.index(answer_end))
    except ValueError:
        return row[:-3] + (None, None)
nlp = spacy.load('en', parser=False)

In [ ]:
with open('../SQuAD/meta.msgpack', 'rb') as f:
    meta = msgpack.load(f, encoding='utf8')
with open('../SQuAD/data.msgpack', 'rb') as f:
    data = msgpack.load(f, encoding='utf8')

In [2]:
dev_file = '../SQuAD/dev-v1.1.json'
dev = flatten_json(dev_file, 'dev')
dev_y = [x[-1] for x in data['dev']]

NameError: name 'json' is not defined

In [140]:
def unveil_detail(case_id):

    # check if the id match 
    assert dev[case_id][0] == data['dev'][case_id][0]
    dev_ann = annotate(dev[case_id])
    context = dev_ann[1]
    c_pos = dev_ann[3]
    c_ner = dev_ann[4]
    c_iob_np = dev_ann[5]
    c_iob_ner = dev_ann[6]
    c_part_ner = dev_ann[7]
    q_pos = dev_ann[8]
    q_ner = dev_ann[9]
    q_iob_np = dev_ann[10]
    q_iob_ner = dev_ann[11]
    question = dev_ann[12]
    context_para = dev_ann[13]
    answer = dev_ann[15]
    question_df = pd.DataFrame(np.column_stack([question,q_pos,q_iob_np,q_ner,q_iob_ner]))
    context_df = pd.DataFrame(np.column_stack([context,c_pos,c_iob_np,c_ner,c_iob_ner, c_part_ner]))

    return question_df, context_df, context_para

## error analysis

In [141]:
question_df, context_df,context_para = unveil_detail(3)

TypeError: list indices must be integers or slices, not str

In [98]:
print(question_df.to_string())

              0    1     2     3      4
0         which  WDT  o_np        o_ner
1           nfl  NNP  b_np   ORG  b_ner
2          team   NN  i_np        o_ner
3   represented  VBD  o_np        o_ner
4           the   DT  o_np        o_ner
5           nfc  NNP  b_np  NORP  b_ner
6            at   IN  o_np        o_ner
7         super  NNP  b_np   ORG  b_ner
8          bowl  NNP  i_np   ORG  i_ner
9            50   CD  o_np        o_ner
10            ?    .  o_np        o_ner


In [99]:
print(context_para)

Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [100]:
print(context_df.to_string())

               0      1     2            3      4      5
0          super    NNP  b_np          ORG  b_ner  i_ner
1           bowl    NNP  i_np          ORG  i_ner  i_ner
2             50     CD  o_np               o_ner  i_ner
3            was    VBD  o_np               o_ner  o_ner
4             an     DT  o_np               o_ner  o_ner
5       american     JJ  o_np         NORP  b_ner  i_ner
6       football     NN  b_np               o_ner  i_ner
7           game     NN  i_np               o_ner  o_ner
8             to     TO  o_np               o_ner  o_ner
9      determine     VB  o_np               o_ner  o_ner
10           the     DT  o_np               o_ner  o_ner
11      champion     NN  b_np               o_ner  o_ner
12            of     IN  o_np               o_ner  o_ner
13           the     DT  o_np          ORG  b_ner  o_ner
14      national    NNP  b_np          ORG  i_ner  i_ner
15      football    NNP  i_np          ORG  i_ner  i_ner
16        league    NNP  i_np  